In [8]:
import pandas as pd

In [10]:
#importação de dados
df_original = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/DADOS_SISREG.csv', 
    dtype={'CNES_EXECUTANTE':'str','IBGE_USUARIO':'str','CEP_USUARIO':'str', 'CEP_UNID_EXECUTANTE':'str'}) 
cep = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_BR.txt', dtype=str, sep="	", header=None)
cep.columns = ['CEP','UF','CIDADE','BAIRRO','LOGRADOURO']  #http://cep.la/baixar

In [1]:
cep['CEP'] = cep.astype(str('CEP').zfill(8))

NameError: name 'cep' is not defined